In [1]:
import pandas as pd

In [2]:
users = pd.read_csv("users.csv")
users

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,tj,TJ Holowaychuk,APEX,"London, UK",tj@apex.sh,False,NaN,326,50841,45,2008-09-18T22:37:28Z
1,kunal-kushwaha,Kunal Kushwaha,CIVO,"London, UK",contact@techwithkunal.com,False,Field CTO @civo. Founder @WeMakeDevs,56,30745,0,2018-08-25T15:01:59Z
2,angelabauer,Angela Yu,APPBREWERY,"London, UK",NaN,False,Medical doctor. Developer. Founder of the App ...,975,18607,0,2014-09-16T21:03:28Z
3,jlord,Jessica Lord,NaN,"London, England",NaN,False,Urban designer turned open source developer. S...,197,16357,30,2012-01-05T03:24:52Z
4,Elfocrash,Nick Chapsas,NaN,"London, UK",nick@nickchapsas.com,False,Content creator for C# & .NET,50,10561,0,2014-07-18T09:32:23Z
...,...,...,...,...,...,...,...,...,...,...,...
296,AlexsJones,Alex Jones,AWS,London,alex@k8sgpt.ai,False,Principal Engineer @aws,252,512,60,2011-12-02T11:40:05Z
297,Rigellute,Alexander Keliris,KELIRIS CONSULTING,London,rigellute@gmail.com,False,Software consultant. Author of spotify-tui. Lo...,91,510,6,2015-04-28T08:47:01Z
298,benyaminahmed,Benyamin Ahmed,NaN,"London, UK",NaN,False,NaN,10,510,0,2018-02-11T11:42:12Z
299,ryancdotorg,Ryan Castellucci,NaN,"London, UK",github-1b66+210325@ryanc.org,True,Hacker. Bot slayer. Ex-sysadmin. Bureaucramanc...,133,508,10,2011-03-08T00:48:52Z


In [3]:
repos = pd.read_csv("repositories.csv")
repos

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,tj,tj/git-extras,2010-08-04T16:32:07Z,17309,210,Shell,True,True,mit
1,tj,tj/n,2011-01-05T14:53:19Z,18849,203,Shell,True,True,mit
2,tj,tj/commander.js,2011-08-14T21:33:58Z,26709,232,JavaScript,True,True,mit
3,tj,tj/node-migrate,2011-04-24T21:00:22Z,1542,18,JavaScript,True,True,mit
4,tj,tj/go-naturaldate,2019-11-26T14:35:09Z,300,9,Go,True,True,mit
...,...,...,...,...,...,...,...,...,...
37917,Neha,Neha/reactjs-demo,2016-05-29T01:44:15Z,4,1,JavaScript,True,True,NaN
37918,Neha,Neha/webcomponents,2016-05-07T19:38:55Z,1,1,JavaScript,True,True,NaN
37919,Neha,Neha/webcomponents-demo,2016-05-05T18:54:13Z,2,2,HTML,True,True,NaN
37920,Neha,Neha/React-skeleton,2016-04-16T20:20:56Z,1,1,JavaScript,True,True,NaN


In [4]:
users.created_at = pd.to_datetime(users.created_at)
repos.created_at = pd.to_datetime(repos.created_at)

Who are the top 5 users in London with the highest number of followers? List their login in order, comma-separated.

In [5]:
",".join(users.nlargest(5, "followers")["login"])

'tj,kunal-kushwaha,angelabauer,jlord,Elfocrash'

Who are the 5 earliest registered GitHub users in London? List their login in ascending order of created_at, comma-separated.

In [6]:
",".join(users.nsmallest(5, "created_at")["login"])

'aslakhellesoy,tomstuart,sole,javache,philhawksworth'

What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [7]:
",".join(repos["license_name"].dropna().value_counts().head(3).index)

'mit,apache-2.0,other'

Which company do the majority of these developers work at?

In [8]:
users["company"].mode()[0]

'FACEBOOK'

Which programming language is most popular among these users?

In [9]:
repos["language"].mode()[0]

'JavaScript'

Which programming language is the second most popular among users who joined after 2020?

In [10]:
pd.merge(users[users["created_at"] >= "2020-01-01"], repos, on="login")[
    "language"
].value_counts().index[1]

'JavaScript'

Which language has the highest average number of stars per repository?

In [11]:
repos.groupby("language")["stargazers_count"].mean().idxmax()

'Reason'

Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [12]:
users["leader_strength"] = users["followers"] / (1 + users["following"])
",".join(users.nlargest(5, "leader_strength")["login"])

'kunal-kushwaha,angelabauer,Elfocrash,LaravelDaily,sonnysangha'

What is the correlation between the number of followers and the number of public repositories among users in London?

In [13]:
round(users["followers"].corr(users["public_repos"]), 3)

np.float64(0.149)

 Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [14]:
import statsmodels.api as sm

# Define the independent variable (public_repos) and dependent variable (followers)
X = users["public_repos"]
y = users["followers"]

# Add a constant to the independent variable
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())

# Get the coefficient for public_repos
round(model.params["public_repos"], 3)

                            OLS Regression Results                            
Dep. Variable:              followers   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     6.832
Date:                Sat, 02 Nov 2024   Prob (F-statistic):            0.00941
Time:                        15:18:40   Log-Likelihood:                -2907.2
No. Observations:                 301   AIC:                             5818.
Df Residuals:                     299   BIC:                             5826.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1363.5427    278.391      4.898   

np.float64(3.247)

Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [15]:
round(repos[["has_projects", "has_wiki"]].corr().iloc[0, 1], 3)

np.float64(0.442)

Do hireable users follow more people than those who are not hireable?

In [16]:
round(
    users[users["hireable"]]["following"].mean()
    - users[~users["hireable"]]["following"].mean(),
    3,
)

np.float64(1064.155)

Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode characters) with followers? (Ignore people without bios)

In [17]:
import statsmodels.api as sm

# Filter out users without bios
users_with_bio = users.dropna(subset=["bio"])

# Define the independent variable (bio_word_count) and dependent variable (followers)
X = users_with_bio["bio"].str.split().str.len()
y = users_with_bio["followers"]

# Add a constant to the independent variable
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the regression slope of followers on bio word count
round(model.params["bio"], 3)

np.float64(3.423)

Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [18]:
",".join(repos[repos["created_at"].dt.dayofweek >= 5].groupby("login").size().nlargest(5).index)

'praveenscience,passy,CodeMaster7000,mattdesl,UKVeteran'

Do people who are hireable share their email addresses more often?

In [19]:
round(
    users[users["hireable"]]["email"].notna().mean()
    - users[~users["hireable"]]["email"].notna().mean(),
    3,
)

np.float64(0.075)

Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [20]:
# Drop rows with missing names
users_with_names = users.dropna(subset=["name"]).copy()

# Extract the last word in the name as the surname
users_with_names["surname"] = users_with_names["name"].str.split().str[-1]

# Find the most common surname(s)
surname_counts = users_with_names["surname"].value_counts()
most_common_surnames = surname_counts[surname_counts == surname_counts.max()].index

# List the most common surname(s) comma-separated
",".join(sorted(most_common_surnames))

'Appleton,Fuller,Greenfeld,Jackson,Li,Williams'